In [1]:
import numpy as np

In [35]:
test_block = np.random.randint(0,2,(3,3))
test_block

array([[0, 0, 0],
       [1, 1, 0],
       [1, 1, 0]])

In [36]:
print(first_idx := (test_block != 0).argmax())

3


In [38]:
test_block.shape

(3, 3)

In [41]:
x,y = int(first_idx / test_block.shape[1]), first_idx % test_block.shape[0]
print(x,y)
test_block[x][y]

1 0


1

In [42]:
test_block = np.random.randint(0,1,(3,3))
test_block

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [43]:
print(first_idx := (test_block != 0).argmax())

0


In [44]:
x,y = int(first_idx / test_block.shape[1]), first_idx % test_block.shape[0]
print(x,y)
test_block[x][y]

0 0


0